# Million Song Dataset - MillionSongSubset
You can download the [sample dataset here](http://millionsongdataset.com/pages/getting-dataset/), which contains 10,000 songs (1%, 1.8 gb) selected at random.

In [1]:
import pandas as pd
import os
import glob
import h5py
import numpy as np
import shutil
import json
import sqlite3

## MSD - Main Subset File

## Set up directory

In [2]:
ROOT_DIR = os.getcwd()
# source_directory = r"C:\Users\Nicholas\Downloads\MillionSongSubset\A"
source_directory = r"C:\Users\Nicholas\Downloads\MillionSongSubset\B"
destination_directory = os.path.join(ROOT_DIR, 'sample_data', 'MSD-MAIN')

## Step 1: Move h5 files into 1 main folder

In [248]:
# Iterate over the first level of folders (A to Z)
for first_level_folder in range(ord('A'), ord('Z')+1):
    first_level_folder = chr(first_level_folder)
    first_level_path = os.path.join(source_directory, first_level_folder)

    # Iterate over the second level of folders (A to Z)
    for second_level_folder in range(ord('A'), ord('Z')+1):
        second_level_folder = chr(second_level_folder)
        second_level_path = os.path.join(first_level_path, second_level_folder)

        # Iterate over the h5 files in the second level folder
        for file_path in glob.glob(os.path.join(second_level_path, "*.h5")):
            # Move the h5 file to the destination directory
            shutil.move(file_path, destination_directory)

In [3]:
file_list = glob.glob(destination_directory + "/*.h5") # Include slash or it will search in the wrong directory!!
len(file_list)

10000

## Get song data

In [4]:
# List to store dataframes
dfs = []

# Iterate over the h5 files in the directory
for file_path in file_list:
    h5_file = h5py.File(file_path, 'r')

    # Extract the data from h5_file['analysis']['songs']
    data = h5_file['analysis']['songs'][()]

    # Convert the structured array to a DataFrame
    df = pd.DataFrame(data)

    # Append the dataframe to the list
    dfs.append(df)

    # Close the h5 file
    h5_file.close()

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dfs, ignore_index=True)
display(len(combined_df))
display(combined_df.head())

10000

,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,...,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,track_id
0,22050,b'a222795e07cd65b7a530f1346f520649',0.0,218.93179,0.247,0.0,0,0,0,0,...,1,0.736,-11.197,0,0.636,218.932,92.198,4,0.778,b'TRAAAAW128F429D538'
1,22050,b'bb9771eeef3d5b204a3c55e690f52a91',0.0,148.03546,0.148,0.0,0,0,0,0,...,6,0.169,-9.843,0,0.430,137.915,121.274,4,0.384,b'TRAAABD128F429CF47'
2,22050,b'fa329738005ca53715d9f7381a0d1fe3',0.0,177.47546,0.282,0.0,0,0,0,0,...,8,0.643,-9.689,1,0.565,172.304,100.070,1,0.000,b'TRAAADZ128F9348C2E'
3,22050,b'43cd1abd45d5a2dda16a3c65b4963bd4',0.0,233.40363,0.000,0.0,0,0,0,0,...,0,0.751,-9.013,1,0.749,217.124,119.293,4,0.000,b'TRAAAEF128F4273421'
4,22050,b'580a8fe08ef0f1c7734b84547d7a8bc7',0.0,209.60608,0.066,0.0,0,0,0,0,...,2,0.092,-4.501,1,0.371,198.699,129.738,4,0.562,b'TRAAAFD128F92F423A'


In [5]:
combined_df.describe()

,analysis_sample_rate,danceability,duration,end_of_fade_in,energy,idx_bars_confidence,idx_bars_start,idx_beats_confidence,idx_beats_start,idx_sections_confidence,...,idx_tatums_start,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence
count,10000.0,10000.0,10000.000000,10000.000000,10000.0,10000.0,10000.0,10000.0,10000.0,10000.0,...,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,22050.0,0.0,238.507518,0.758616,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.276100,0.449573,-10.485668,0.691100,0.477784,229.975465,122.915449,3.564800,0.509937
std,0.0,0.0,114.137514,1.867952,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.554087,0.274974,5.399788,0.462063,0.191254,112.195735,35.184412,1.266239,0.373409
min,22050.0,0.0,1.044440,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,-51.643000,0.000000,0.000000,1.044000,0.000000,0.000000,0.000000
25%,22050.0,0.0,176.032200,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.000000,0.225000,-13.163250,0.000000,0.360000,168.856000,96.965750,3.000000,0.097750
50%,22050.0,0.0,223.059140,0.199000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.000000,0.469000,-9.380000,1.000000,0.487000,213.879000,120.161000,4.000000,0.551000
75%,22050.0,0.0,276.375060,0.421000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.000000,0.659000,-6.532500,1.000000,0.606000,266.292000,144.013250,4.000000,0.864000
max,22050.0,0.0,1819.767710,43.119000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,11.000000,1.000000,0.566000,1.000000,1.000000,1813.426000,262.828000,7.000000,1.000000


In [251]:
# Save dataframe to csv
combined_df.to_csv("MSD_SONG_DATA.csv", index=False)

## Last.fm Dataset
The Last.fm dataset consists of two kinds of data at the song level: tags and similar songs. If you are familiar with the Last.fm API, it corresponds to the track methods 'getTopTags' and 'getSimilar'.

In [14]:
# source_directory = r"C:\Users\Nicholas\Downloads\lastfm_subset\lastfm_subset\A"
source_directory = r"C:\Users\Nicholas\Downloads\lastfm_subset\lastfm_subset\B"
destination_directory = os.path.join(ROOT_DIR, 'sample_data', 'Lastfm')

In [15]:
# Iterate over the first level of folders (A to Z)
for first_level_folder in range(ord('A'), ord('Z')+1):
    first_level_folder = chr(first_level_folder)
    first_level_path = os.path.join(source_directory, first_level_folder)

    # Iterate over the second level of folders (A to Z)
    for second_level_folder in range(ord('A'), ord('Z')+1):
        second_level_folder = chr(second_level_folder)
        second_level_path = os.path.join(first_level_path, second_level_folder)

        # Iterate over the h5 files in the second level folder
        for file_path in glob.glob(os.path.join(second_level_path, "*.json")):
            # Move the h5 file to the destination directory
            shutil.move(file_path, destination_directory)

In [18]:
lstfm_file_list = glob.glob(destination_directory + "/*.json") # Include slash or it will search in the wrong directory!!
len(lstfm_file_list)

9330

## Get data from json files and put into pandas data frame

In [19]:
lstfm_file_list[0]

'C:\\Users\\Nicholas\\Documents\\Northwestern\\MSDS495\\MSDS495-Capstone\\sample_data\\Lastfm\\TRAAAAW128F429D538.json'

In [23]:
data = []

for file_path in  lstfm_file_list:
    with open(file_path, 'r') as file:
                json_data = json.load(file)
                data.append(json_data)
                
data[:2]

[{'artist': 'Casual',
  'timestamp': '2011-08-02 20:13:25.674526',
  'similars': [['TRABACN128F425B784', 0.871737],
   ['TRIAINV12903CB4943', 0.751301],
   ['TRJYGLF12903CB4952', 0.751052],
   ['TRVNXHF128F93134F0', 0.582662],
   ['TRWJMMB128F429D550', 0.508661],
   ['TRUEULA128F425B7DB', 0.404535],
   ['TRIEUIZ128F427973B', 0.028357],
   ['TRWNLAW128F9364539', 0.0278891],
   ['TRWIMTU128F148D424', 0.0227976],
   ['TRPTKMG128F930E997', 0.0198059],
   ['TRQIPME12903CE0A48', 0.0161872],
   ['TRAOGGC128F4215EB5', 0.0158649],
   ['TROIDMT128F428ED1F', 0.0158644],
   ['TRAKXGR128F428686F', 0.0156715],
   ['TRQKESV128F428B3AC', 0.0155405],
   ['TRCRZMU128F931E6C0', 0.0154048],
   ['TRKDOBN12903D0C783', 0.0152418],
   ['TRXVODK12903D0C786', 0.0152418],
   ['TRIVHFX12903D0CCF0', 0.0145524],
   ['TRUZARV12903D0CCE5', 0.0145524],
   ['TRTSCMP12903CB5891', 0.0143529],
   ['TRWGIRJ128F425DCB2', 0.0141029],
   ['TRZEKXA128F9342FFC', 0.0139119],
   ['TRHVRPE128F934DB59', 0.0134494],
   ['TRZYTKI128F

In [24]:
lastfm_df = pd.DataFrame(data)
lastfm_df

,artist,timestamp,similars,tags,track_id,title
0,Casual,2011-08-02 20:13:25.674526,"[[TRABACN128F425B784, 0.871737], [TRIAINV12903...","[[Bay Area, 100], [hieroglyiphics, 100], [clas...",TRAAAAW128F429D538,I Didn't Mean To
1,The Box Tops,2011-08-12 04:04:21.577908,"[[TRBRSNN128F4284E8C, 1], [TRHYADV128F4285EC1,...","[[60s, 100], [soul, 52], [pop, 41], [rock, 35]...",TRAAABD128F429CF47,Soul Deep
2,La Sonora Santanera,2011-08-09 18:24:38.890272,"[[TRPHLNJ128F9348769, 1], [TRVWNCB128F9343925,...","[[latin, 100]]",TRAAADZ128F9348C2E,Amor De Cabaret
3,Adam Ant,2011-08-11 21:57:49.876264,"[[TRWXGJB12903CCFE02, 1], [TRKDLTP128F427F677,...","[[new wave, 100], [80s, 66], [freedom, 33], [s...",TRAAAEF128F4273421,Something Girls
4,Gob,2011-08-09 01:59:41.352247,"[[TRTOVWD128F92F4227, 1], [TRUXNUD128F92F41D0,...","[[punk rock, 100], [punk, 60]]",TRAAAFD128F92F423A,Face the Ashes
...,...,...,...,...,...,...
9325,Moonspell,2011-08-02 19:50:29.556601,"[[TRRSECQ12903CF8927, 1], [TRYAZQO12903CF8928,...","[[Gothic Metal, 100], [moonspell, 57], [doom m...",TRBIJMU12903CF892B,The Hanged Man
9326,Danny Williams,2011-08-03 11:10:33.648275,"[[TRPKQSS128F9320C71, 0.747832], [TRJQUTO128F4...","[[oldies, 100], [feelings, 100]]",TRBIJNF128F14815A7,The Wonderful World Of The Young
9327,Winston Reedy,2011-08-11 11:53:48.471871,[],[],TRBIJNK128F93093EC,Sentimental Man
9328,"Myrick ""Freeze"" Guillory",2011-08-11 02:01:49.987232,[],"[[zydeco, 100], [novea zydeco, 100]]",TRBIJRN128F425F3DD,Zydeco In D-Minor


In [25]:
# Save dataframe to csv
lastfm_df.to_csv("LASTFM_DATA.csv", index=False)

## musiXmatch Dataset

In [27]:
DB_FILE_PATH = r'C:\Users\Nicholas\Documents\Northwestern\MSDS495\MSDS495-Capstone\mxm_dataset.db'
connection = sqlite3.connect(DB_FILE_PATH)

In [28]:
cursor = connection.cursor()

### Get the table names in database

In [29]:
cursor.execute('''
SELECT name 
FROM sqlite_master 
WHERE type='table';
''')
tables = cursor.fetchall()

In [30]:
# Print table names
for table in tables:
    print(table[0])

words
lyrics


## Sample query
See [README](https://github.com/tbertinmahieux/MSongsDB/blob/master/Tasks_Demos/Lyrics/README.txt) for details
- table 'lyrics' contains 5 columns, see below
   - column 'track_id' -> as usual, track id from the MSD
   - column 'mxm_tid' -> track ID from musiXmatch
   - column 'word' -> a word that is also in the 'words' table
   - column 'count' -> word count for the word
   - column 'is_test' -> 0 if this example is from the train set, 1 if test

In [32]:
# Sample query
cursor.execute('''
SELECT * 
FROM lyrics 
LIMIT 50;
''')
rows = cursor.fetchall()

# Print the output
for row in rows:
    print(row)

('TRAAAAV128F421A322', 4623710, 'i', 6, 0)
('TRAAAAV128F421A322', 4623710, 'the', 4, 0)
('TRAAAAV128F421A322', 4623710, 'you', 2, 0)
('TRAAAAV128F421A322', 4623710, 'to', 2, 0)
('TRAAAAV128F421A322', 4623710, 'and', 5, 0)
('TRAAAAV128F421A322', 4623710, 'a', 3, 0)
('TRAAAAV128F421A322', 4623710, 'me', 1, 0)
('TRAAAAV128F421A322', 4623710, 'it', 1, 0)
('TRAAAAV128F421A322', 4623710, 'my', 1, 0)
('TRAAAAV128F421A322', 4623710, 'is', 2, 0)
('TRAAAAV128F421A322', 4623710, 'of', 3, 0)
('TRAAAAV128F421A322', 4623710, 'your', 1, 0)
('TRAAAAV128F421A322', 4623710, 'that', 1, 0)
('TRAAAAV128F421A322', 4623710, 'are', 2, 0)
('TRAAAAV128F421A322', 4623710, 'we', 2, 0)
('TRAAAAV128F421A322', 4623710, 'am', 2, 0)
('TRAAAAV128F421A322', 4623710, 'will', 2, 0)
('TRAAAAV128F421A322', 4623710, 'for', 4, 0)
('TRAAAAV128F421A322', 4623710, 'be', 1, 0)
('TRAAAAV128F421A322', 4623710, 'have', 2, 0)
('TRAAAAV128F421A322', 4623710, 'so', 1, 0)
('TRAAAAV128F421A322', 4623710, 'this', 1, 0)
('TRAAAAV128F421A32

In [34]:
# Close the connection
connection.close()